In [2]:
import json
import pandas as pd
import numpy as np

#add all simulated data point jsons into this list
simulation_files = ['simulation_DIST_6000 sipmle.json','simulation_DIST_6000 sipmle_lowRR.json','simulation_DIST_6000-12000 sipmle constnat mole frac.json','simulation_DIST_simple 6000-6600.json','simulation_DIST_12000-18000 .01 RR.json']

values = [] #feed_a, feed_b, feed_c, dist_a, dist_b, dist_c, bott_a, bott_b, bott_c, cond_duty, reb_duty, # trays, feed tray,dist_rate,reflux_ratio
components = ['N-BUTANE','N-PENTAN','N-HEXANE']
block_vals = ["Investment Cost","Operating Cost Reboiler","Operating Cost Condenser",'DIST_RATE','REFLUX_RATIO','COND-DUTY','REB-DUTY']
for f in simulation_files:
    with open(f,"r") as file:
        jsonData = json.load(file)
        
        for i in range(1,len(jsonData)+1):
            values_temp = [] 
            for j in components:
                #print(jsonData[str(i)]['FEED']['MOLEFLOW'][j])
                values_temp.append(jsonData[str(i)]['FEED']['MOLEFLOW'][j])
            for j in components:
                values_temp.append(jsonData[str(i)]['DIST']['MOLEFLOW'][j])
            for j in components:
                values_temp.append(jsonData[str(i)]['BOTT']['MOLEFLOW'][j])
            for j in block_vals:
                values_temp.append(jsonData[str(i)]['B1'][j])
            values.append(values_temp)
            
df = pd.DataFrame(values, columns=['FEED_BUT','FEED_PEN','FEED_HEX','DIST_BUT','DIST_PEN','DIST_HEX','BOTT_BUT',
                                             'BOTT_PEN','BOTT_HEX',"Investment Cost","Operating Cost Reboiler","Operating Cost Condenser",'DIST_RATE','REF_RAT','COND_DUTY','REB_DUTY'])

df = df[~df['Investment Cost'].isin(['Returned None'])] #getting rid of simulations that had errors
df['Operating Cost Reboiler'] = 1.07 * np.array(df['Operating Cost Reboiler']) / 1e4 #scaling units to be 10,000$/year 1.07 scaler is for converting euro to usd
df['Investment Cost'] = 1.07 * np.array(df['Investment Cost']) / 1e4
df['Operating Cost Condenser'] = 1.07 * np.array(df['Operating Cost Condenser']) / 1e4
df['Operating Cost'] = np.array(df['Operating Cost Condenser']) + np.array(df['Operating Cost Reboiler'])
df['Reflux Rate'] = np.array(df['REF_RAT'])*np.array(df['DIST_RATE'])
df = df[df['Investment Cost'] <= 600] #removing outliers

#either add path to Solving Formulation folders, or copy and paste datarame over there.
df.to_csv('dataframe')
df.to_csv('...\\Solving Formulation\\dataframe')

OSError: Cannot save file into a non-existent directory: '...\Training NN'